In [116]:
import SimFunctions
import SimClasses
import SimRNG
import SimRNG_Modified
import simpy
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy.stats import probplot, kstest
import pickle
warnings.filterwarnings("ignore")
# fix random number seed
np.random.seed(1)


# Classes

In [117]:
class Station:
    def __init__(self, station_id, level, capacity):
        self.id = station_id
        self.level = level
        self.capacity = capacity
        self.bikes = {}
        self.bike_list = []
        for i in range(level):
            bike_id = f"{station_id}-{i+1}"  # create unique bike ID
            self.bikes[bike_id] = True  # mark bike as available
            self.bike_list.append(bike_id)
            
    def rent_bike(self):
        # Request a bike from the station
        if self.level > 0:
            if self.bike_list:
                random_index = np.random.randint(0, len(self.bike_list))
                bike_id = self.bike_list.pop(random_index)
                self.level -= 1
                return bike_id
        return None
    
    def return_bike(self, bike_id):
        # # Return a bike to the station
        if self.level < self.capacity:
            self.bike_list.append(bike_id)
            self.level += 1

    def Get_Bike_List(self):
        return self.bike_list
    
class Customer:
    def __init__(self, customer_id, start_station, bike = None):
        self.customer_id = customer_id
        self.start_station = start_station
        self.end_id = 0
        self.station_level = 0
        self.bike = bike

    def rent_bike(self):
        station = StationDict[self.start_station]
        print(f"Customer Arrives at S{station.id} with Level: {station.level}")

        if station.level > 0:
            self.bike = station.rent_bike()
            self.station_level = station.level
            print("(Customer Rent Bikes) Customer ID: {} | Bike ID {} || From: S{} -> Remaining Level: {}".format(self.customer_id,
                                                                                                            self.bike,
                                                                                                            self.start_station,
                                                                                                            self.station_level))
            SimFunctions.Schedule(Calendar, "Departure", 0.1)
        else:
            print(f"    Customer {self.customer_id} CANNOT RENT BIKE -- EMPTY STATION {self.start_station} w/ level {self.station_level} -- EMPTY")
            
    def return_bike(self, end_station, bike):
        destination_station = StationDict[end_station]
        destination_station.return_bike(bike)
        self.station_level = destination_station.level
        
        

# Functions

In [118]:
def Start():
    SimFunctions.Schedule(Calendar, "Customer_Arrival", SimRNG_Modified.Expon(ZSimRNG, 0, 1))
    
def NextCustomerID():
    if not hasattr(NextCustomerID, "counter"):
        NextCustomerID.counter = 0
    NextCustomerID.counter += 1
    return NextCustomerID.counter

## Probabilistic Functions

In [119]:

def Customer_Arrival_Rate(station_id):
    U = SimRNG_Modified.Uniform(ZSimRNG, 0, 1, 5)
    if U < (1/3):
        station_id = 1
    elif U < (2/3) and U > (1/3):
        station_id = 2
    else:
        station_id = 3
    return station_id


def Destination(customer, retrial=0):
    U = SimRNG_Modified.Uniform(ZSimRNG, 0, 1, 3)
    if U < (1/3):
        end_id = 1
    elif U < (2/3) and U > (1/3):
        end_id = 2
    else:
        end_id = 3

    end_station = StationDict[end_id]
    if customer.bike is None:
        print(" Customer ID: {} Leaves from empty | From: {} -> Remaining Level {}".format(customer.customer_id,
                                                                                           customer.start_station,
                                                                                           customer.station_level))
    elif retrial == 1:
        pass

    elif retrial == 0 and customer.bike is not None:
        print(" (DEPARTING) Customer ID: {} | Bike ID: {} || From: S{} -> Level {} | To: S{} -> Level {}".format(customer.customer_id,
                                                                                                                 customer.bike,
                                                                                                                 customer.start_station,
                                                                                                                 customer.station_level,
                                                                                                                 end_station.id,
                                                                                                                 end_station.level))
        # print("-------------------------------------------------------------------------")
    return end_id


## Trip Process Functions

In [120]:
def Customer_Arrival(station_id, empty_error, CustomerList):
    SimFunctions.Schedule(Calendar, "Customer_Arrival", 
                          SimRNG_Modified.Expon(ZSimRNG, 0.5, 1))
    station = StationDict[station_id]
    customer_id = NextCustomerID()
    customer = Customer(customer_id, station_id)
    customer.start_station = station_id
    customer.station_level = station.level

################################################################
# STATION EMPTY
################################################################
    if customer.station_level == 0:
        print(f"    EMPTY -- Customer {customer.customer_id} CANNOT RENT BIKE | S{customer.start_station} -> level {customer.station_level} -- EMPTY")
        empty_error += 1
    else:
        CustomerList.append(customer)
        customer.rent_bike()

    return Customer_Arrival_Rate(station_id), empty_error, customer


def Departure(station_id, customer):
    station = StationDict[station_id]
    end_id = Destination(customer, 0)
    customer.end_id = end_id
    SimFunctions.Schedule(Calendar, "Bike_Arrival",
                          15 * SimRNG_Modified.Lognormal(ZSimRNG, 0.5, np.sqrt(0.066), 4))
    return end_id


def Bike_Arrival(end_id, Full_Error, CustomerList, temp_customer):
    
    end_station = StationDict[end_id] 
    
    if end_station.level < end_station.capacity:
        for customer in CustomerList:
            if customer.end_id == end_station.id and customer.bike is not None:
                customer.return_bike(customer.end_id, customer.bike)
                print("     (BIKE RETURNED) Customer ID: {} | Bike ID: {} || To: S{} -> Level {} | From: S{}".format(customer.customer_id,
                                                                                                                        customer.bike,
                                                                                                                        end_station.id,
                                                                                                                        end_station.level,
                                                                                                                        customer.start_station
                                                                                                                        ))
                CustomerList.remove(customer)
                return Full_Error

################################################################
#STATION FULL
################################################################
    else:
        for customer in CustomerList:
            customer_end_station = StationDict[customer.start_station]
            if customer.end_id == customer_end_station.id and customer_end_station.level >= customer_end_station.capacity:
                temp_customer = customer
                print(f"     FULL --  Customer ID: {customer.customer_id} || To: S{end_station.id} -> Level {end_station.level} | Capacity {end_station.capacity} || From: S{customer.start_station} | Bike ID: {customer.bike}")
                Full_Error += 1
                SimFunctions.Schedule(Calendar, "Retrial", SimRNG_Modified.Expon(ZSimRNG, 2, 5))
                Retrial(end_id, Full_Error, CustomerList, temp_customer=temp_customer)
                return Full_Error
            else:
                pass
    
    return Full_Error


def Retrial(end_id, Full_Error, CustomerList, temp_customer):
    customer = temp_customer
    for customer in CustomerList:
        customer_end_station = StationDict[customer.start_station]
        if customer.end_id == customer_end_station.id and customer_end_station.level >= customer_end_station.capacity:
            pre_end_id = customer.end_id
            start_id = customer.start_station
            end_id = pre_end_id
            condition = False

            while not condition:
                if pre_end_id == end_id or start_id == end_id:
                    end_id = Destination(customer, retrial=1)
                else:
                    customer.end_id = end_id
                    condition = True

            print(f"    (NEW STATION) Customer ID: {customer.customer_id} tries new station S{customer.end_id}")
            SimFunctions.Schedule(Calendar, "Bike_Arrival", 
                                15 *SimRNG_Modified.Lognormal(ZSimRNG, 0.5, np.sqrt(0.066), 4))

            return Full_Error
    return Full_Error

# Simulation

In [121]:
NextCustomerID.counter = 0

ZSimRNG = SimRNG_Modified.InitializeRNSeed()

Calendar = SimClasses.EventCalendar()

Queue = SimClasses.FIFOQueue()
Wait = SimClasses.DTStat()
Bikes = SimClasses.Resource()

TheCTStats = []
TheDTStats = []
TheQueues = []
TheResources = []

CustomerList = []
Full_Error_list = []
Empty_Error_list = []

MeanTBA = 0.5
MeanTR = 0.5
MeanST = 0.5

Phases = 3
RunLength = 50
WarmUp = 5
end_id = 1


for days in range(0, 5, 1):
    Full_Error = 0
    Empty_Error = 0
    
    inital_station_id = np.random.random_integers(1, 3)
    station_id = inital_station_id
    
    # Initialize the stations and create a dictionary mapping station IDs to Station instances
    Stations = [Station(station_id=1, level=5, capacity=5), Station(station_id=2, level=35, capacity=40), Station(station_id=3, level=25, capacity=25)]
    StationDict = {station.id: station for station in Stations}
    for station_id, station in StationDict.items():
        print(f"Station {station_id} Inital Bike List: {station.Get_Bike_List()}")
        
    SimFunctions.SimFunctionsInit(Calendar, TheQueues, TheCTStats, TheDTStats, TheResources)
    SimFunctions.Schedule(Calendar, "Start", SimRNG_Modified.Expon(ZSimRNG, 0, 1))
        
    NextEvent = Calendar.Remove()
    SimClasses.Clock = NextEvent.EventTime
    if NextEvent.EventType == "Start":
        Start()

    for hours in [8, 8.3, 9, 9.3, 10, 10.3, 11, 11.3, 12]:  # 30 min intervals
        print(f"          Clock Hour | {hours}")
        for minutes in [i for i in range(1, 31)]:  # 30 min intervals
            print("             Minute | {}".format(minutes))
            
            NextEvent = Calendar.Remove()
            SimClasses.Clock = NextEvent.EventTime

            if NextEvent.EventType == "Customer_Arrival":
                station_id, Empty_Error, customer = Customer_Arrival(station_id, Empty_Error, CustomerList)

            elif NextEvent.EventType == "Departure":
                end_id = Departure(station_id, customer)
            
            elif NextEvent.EventType == "Bike_Arrival":
                Full_Error = Bike_Arrival(end_id, Full_Error, CustomerList, customer)
            
            elif NextEvent.EventType == "Retrial":
                Full_Error = Retrial(end_id, Full_Error, CustomerList, customer)
               
    for station_id, station in StationDict.items():
        print(f"Station {station_id} Bike List: {station.Get_Bike_List()}")

    print(f"Num of Full Errors: {Full_Error}")
    print(f"Num of Empty Errors: {Empty_Error}")
    Full_Error_list.append(Full_Error)
    Empty_Error_list.append(Empty_Error)
    print(f"End of Day {days}")
    print("--------------------------------------------------------------------------------------------------------------------------------------------------")


Station 1 Inital Bike List: ['1-1', '1-2', '1-3', '1-4', '1-5']
Station 2 Inital Bike List: ['2-1', '2-2', '2-3', '2-4', '2-5', '2-6', '2-7', '2-8', '2-9', '2-10', '2-11', '2-12', '2-13', '2-14', '2-15', '2-16', '2-17', '2-18', '2-19', '2-20', '2-21', '2-22', '2-23', '2-24', '2-25', '2-26', '2-27', '2-28', '2-29', '2-30', '2-31', '2-32', '2-33', '2-34', '2-35']
Station 3 Inital Bike List: ['3-1', '3-2', '3-3', '3-4', '3-5', '3-6', '3-7', '3-8', '3-9', '3-10', '3-11', '3-12', '3-13', '3-14', '3-15', '3-16', '3-17', '3-18', '3-19', '3-20', '3-21', '3-22', '3-23', '3-24', '3-25']
          Clock Hour | 8
             Minute | 1
Customer Arrives at S3 with Level: 25
(Customer Rent Bikes) Customer ID: 1 | Bike ID 3-12 || From: S3 -> Remaining Level: 24
             Minute | 2
 (DEPARTING) Customer ID: 1 | Bike ID: 3-12 || From: S3 -> Level 24 | To: S2 -> Level 35
             Minute | 3
Customer Arrives at S2 with Level: 35
(Customer Rent Bikes) Customer ID: 2 | Bike ID 2-13 || From: S2 -> 

In [122]:
for station_id, station in StationDict.items():
    print(f"Station {station_id} Bike List: {station.Get_Bike_List()}")
    print(station.level)
    print(station.capacity)
print(f"Num of Full Errors: {Full_Error}")
print(f"Num of Empty Errors: {Empty_Error}")


Station 1 Bike List: ['2-27', '3-19']
2
5
Station 2 Bike List: ['2-8', '2-9', '2-11', '2-12', '2-13', '2-16', '2-24', '2-25', '2-29', '2-31', '2-34', '2-35', '3-8', '2-30', '3-10', '2-19', '2-3', '3-6', '2-18', '3-17', '2-30', '3-25', '3-3', '3-11', '3-4', '2-22', '1-2', '2-32', '3-22']
29
40
Station 3 Bike List: ['3-1', '3-2', '3-7', '3-11', '3-20', '3-18', '3-21', '2-5', '3-15', '2-8', '2-28', '1-5', '2-5', '2-33']
14
25
Num of Full Errors: 9
Num of Empty Errors: 7


In [123]:
print(f"Num of Full Errors: {Full_Error_list}")
print(f"Num of Empty Errors: {Empty_Error_list}")

Num of Full Errors: [0, 7, 5, 8, 9]
Num of Empty Errors: [23, 11, 6, 7, 7]
